## Analyze whether SNWD varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='BBBBSSBB'
meas='SNWD'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1022
+-----------------+-----------------+-----------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------+------------+------+--------------------+------+
|          coeff_1|          coeff_2|          coeff_3|           coeff_4|elevation|   label|latitude|longitude|measurement|              res_1|              res_2|              res_3|              res_4|           res_mean|    station|   total_var|undefs|              vector|  year|
+-----------------+-----------------+-----------------+------------------+---------+--------+--------+---------+-----------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------+------------+------+--------------------+------+
|8451.311069620988|619.4263888611853|498.7727679134125|1171.4898316770

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [15]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_1'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_1 FROM weather


,station,year,coeff_1
0,USC00306184,1978.0,8451.311070
1,USC00300785,1987.0,4279.368617
2,USC00301708,1994.0,3102.617385
3,USC00305658,1908.0,4230.698929
4,USC00305925,1960.0,2278.765007


In [16]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_1')
year_station_table.head(10)

station,US1NYOD0014,US1NYOD0023,USC00300015,USC00300018,USC00300368,USC00300379,USC00300424,USC00300500,USC00300608,USC00300668,...,USC00309000,USC00309248,USC00309389,USC00309480,USW00014717,USW00014750,USW00014771,USW00014772,USW00014774,USW00094794
year,,,,,,,,,,,,,,,,,,,,,
1900.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1908.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1926.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1927.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1929.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1931.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1935.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1936.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [17]:
def MS(Mat):
    return np.nanmean(Mat**2)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year)

total MS                   =  4987805.0512
MS removing mean-by-station=  2765617.07227
MS removing mean-by-year   =  3193238.40882


In [18]:
#extract longitude and latitude for each station
feature='coeff_2'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year)

SELECT station, year, coeff_2 FROM weather
total MS                   =  1041796.39176
MS removing mean-by-station=  926429.893424
MS removing mean-by-year   =  358636.448135


In [19]:
#extract longitude and latitude for each station
feature='coeff_3'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_3'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

year_station_table=pdf.pivot(index='year', columns='station', values='coeff_3')
year_station_table.head(10)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year)

SELECT station, year, coeff_3 FROM weather
total MS                   =  614551.497599
MS removing mean-by-station=  563510.858921
MS removing mean-by-year   =  219946.138822


In [20]:
#extract longitude and latitude for each station
feature='coeff_4'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_4'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

year_station_table=pdf.pivot(index='year', columns='station', values='coeff_4')
year_station_table.head(10)

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total MS                   = ',MS(year_station_table)
print 'MS removing mean-by-station= ',MS(tbl_minus_station)
print 'MS removing mean-by-year   = ',MS(tbl_minus_year)

SELECT station, year, coeff_4 FROM weather
total MS                   =  533023.911379
MS removing mean-by-station=  467997.234032
MS removing mean-by-year   =  239259.558137
